In [144]:
##Aquire climate_data

import pandas as pd
import numpy as np
from glob import glob

# 設定檔案名稱模式
file_pattern = "72K220-*.csv"

# 使用 glob 匹配符合檔案名稱模式的所有檔案
file_list = glob(file_pattern)

# 創建一個空的 DataFrame 來存放所有年份的氣候資料
climate_data = pd.DataFrame()

# 迴圈讀取每個檔案並將資料合併到 climate_data
for file in file_list:
    # 從檔案名稱中提取年份
    year = int(file[-8:-4])  # 假設檔案名稱是 "72K220-XXXX.csv"
    
    # 讀取 CSV 檔案
    yearly_climate_data = pd.read_csv(file, skiprows=[1])
    
    # 將年份列加入到資料中
    yearly_climate_data['Year'] = year
    yearly_climate_data['Month'] = yearly_climate_data['觀測時間(month)']
    
    # 合併到 all_climate_data
    climate_data = pd.concat([climate_data, yearly_climate_data], ignore_index=True)

# 顯示合併後的資料
climate_data['Year'] = climate_data['Year'].astype(int)
climate_data['Month'] = climate_data['Month'].astype(int)
print(climate_data)


# 將合併後的資料存為 CSV 檔案
climate_data.to_csv('climate_data.csv', index=False)

     觀測時間(month)  測站氣壓(hPa)  測站最高氣壓(hPa)        測站最高氣壓時間(LST)  測站最低氣壓(hPa)  \
0              1     1014.0       1022.4  2005/01/21 09:00:00       1005.3   
1              2     1013.4       1020.2  2005/02/11 09:00:00       1004.5   
2              3     1013.9       1027.4  2005/03/05 09:00:00       1004.5   
3              4     1009.6       1017.4  2005/04/04 10:00:00       1003.2   
4              5     1004.3       1007.8  2005/05/15 09:00:00        999.2   
..           ...        ...          ...                  ...          ...   
223            8     1000.7       1006.6  2023/08/20 22:08:00        994.0   
224            9     1003.8       1009.4  2023/09/16 08:44:00        979.0   
225           10     1008.7       1014.5  2023/10/22 09:07:00        991.0   
226           11     1013.2       1019.2  2023/11/14 09:45:00       1007.4   
227           12     1012.4       1018.3  2023/12/02 09:32:00       1006.9   

           測站最低氣壓時間(LST)  氣溫(℃)  最高氣溫(℃)          最高氣溫時間(LST)  

In [145]:
##Pre-process climate_data

# 將 2018 年的最低氣溫設為 NaN 
climate_data_fix2018=climate_data
climate_data_fix2018.loc[((climate_data_fix2018['Year'] == 2018) & (climate_data_fix2018['Month'] == 10)), '最低氣溫(℃)'] = np.nan
climate_data_fix2018.loc[((climate_data_fix2018['Year'] == 2015) & (climate_data_fix2018['Month'] == 8)), '最低氣溫(℃)'] = np.nan

# ((climate_data['Year'] == 2018) & (climate_data['觀測時間(month)'] == 10)).to_csv('table.csv', index=False)


# 使用平均值替換 NaN 值
# climate_data_filled=climate_data_fix2018
climate_data_filled = climate_data_fix2018.fillna(climate_data_fix2018.mean())

# 顯示填充後的資料
# print(climate_data_filled)
# # 將填充後的資料存為 CSV 檔案
# climate_data_filled.to_csv('climate_data_filled.csv', index=False)

# 將資料按年度分組
grouped_climate_data = climate_data_filled.groupby("Year")

# 對每個組別進行合併或聚合操作
aggregated_climate_data = grouped_climate_data.agg({
    "測站氣壓(hPa)": "mean",
    "海平面氣壓(hPa)": "mean",
    "測站最高氣壓(hPa)": "max",
    "測站最低氣壓(hPa)": "min",
    "氣溫(℃)": "mean",
    "最高氣溫(℃)": "max",
    "最低氣溫(℃)": "min",
    "風速(m/s)": "mean",
    "最大瞬間風(m/s)": "max",
    "降水量(mm)": "sum",
    "最大日降水量(mm)": "max",
    "地溫0cm": "mean",

})

# 重設索引以使 "Year" 列不再是索引
aggregated_climate_data = aggregated_climate_data.reset_index()

# 顯示合併後的資料
print(aggregated_climate_data)

# 將填充後的資料存為 CSV 檔案
aggregated_climate_data.to_csv('aggregated_climate_data.csv', index=False)


    Year    測站氣壓(hPa)   海平面氣壓(hPa)  測站最高氣壓(hPa)  測站最低氣壓(hPa)      氣溫(℃)  \
0   2005  1008.333333  1017.473611       1027.4        973.1  22.858333   
1   2006  1007.975000  1017.473611       1023.9        979.0  23.383333   
2   2007  1007.883333  1017.473611       1022.2        974.1  23.408333   
3   2008  1008.275000  1017.473611       1021.2        976.2  23.008333   
4   2009  1007.700000  1017.473611       1024.9        964.2  23.316667   
5   2010  1008.633333  1017.473611       1023.5        981.7  23.133333   
6   2011  1008.350000  1017.473611       1022.2        983.8  22.983333   
7   2012  1007.400000  1017.473611       1020.6        982.7  23.441667   
8   2013  1008.416667  1017.473611       1023.0        981.4  23.550000   
9   2014  1008.725000  1017.473611       1022.7        979.1  23.641667   
10  2015  1008.950000  1017.473611       1022.4        959.3  24.016667   
11  2016  1008.708333  1017.473611       1028.9        958.9  24.008333   
12  2017  1009.008333  10

In [146]:
##Aquire rice_data

import pandas as pd

# 讀取CSV檔案
file_path = "臺灣地區稻作種植、收穫面積及產量.csv"
rice_data = pd.read_csv(file_path)


# 顯示稻米資料
print(rice_data)

        年度  地區別 稻作類別     稻作品項      收穫面積         產量
0     2022  嘉義市   水稻  秈糯稻(長糯)         0          0
1     2022  新竹市   水稻  秈糯稻(長糯)         0          0
2     2022  基隆市   水稻  秈糯稻(長糯)         0          0
3     2022  澎湖縣   水稻  秈糯稻(長糯)         0          0
4     2022  花蓮縣   水稻  秈糯稻(長糯)       130        447
...    ...  ...  ...      ...       ...        ...
1926  2005  苗栗縣   水稻   稉稻(蓬萊)  12412.27   48879.11
1927  2005  新竹縣   水稻   稉稻(蓬萊)   7044.89  27296.183
1928  2005  桃園縣   水稻   稉稻(蓬萊)  12347.72  45971.848
1929  2005  宜蘭縣   水稻   稉稻(蓬萊)   8995.46  44142.755
1930  2005  台北縣   水稻   稉稻(蓬萊)    304.81   1218.763

[1931 rows x 6 columns]


In [147]:
##Pre-process rice_data

processed_rice_data=rice_data
processed_rice_data['收穫面積'] = processed_rice_data['收穫面積'].replace('_', 0)
processed_rice_data['產量'] = processed_rice_data['產量'].replace('_', 0)

# 將 '收穫面積' 和 '產量' 轉換為數值型態
processed_rice_data['收穫面積'] = pd.to_numeric(processed_rice_data['收穫面積'], errors='coerce')
processed_rice_data['產量'] = pd.to_numeric(processed_rice_data['產量'], errors='coerce')

# # 顯示稻米資料
# print(processed_rice_data)

# 將 "Year" 欄位轉換為整數
processed_rice_data['年度'] = processed_rice_data['年度'].astype(int)

# 使用平均值替換 NaN 值
rice_data_filled=processed_rice_data
# rice_data_filled = rice_data_filled.fillna(rice_data_filled.mean())

# 將資料按地區篩選
# rice_data_selected = rice_data_filled
rice_data_selected = rice_data_filled.loc[(rice_data_filled['地區別'] == '雲林縣')]

# 將資料按年度分組
grouped_rice_data = rice_data_selected.groupby("年度")

# 對每個組別進行合併或聚合操作
aggregated_rice_data = grouped_rice_data.agg({
	
    "產量": "sum",
})

# 重設索引以使 "Year" 列不再是索引
aggregated_rice_data = aggregated_rice_data.reset_index()

# 顯示合併後的資料
print(aggregated_rice_data)

      年度          產量
0   2005  208289.704
1   2006  225004.805
2   2007  184585.881
3   2008  219487.599
4   2009  241788.425
5   2010  240184.619
6   2011  256466.058
7   2012  268065.774
8   2013  244610.514
9   2014  262391.030
10  2015  256779.924
11  2016  241900.800
12  2017  266549.996
13  2018  208447.762
14  2019  284166.000
15  2020  202351.000
16  2021  266890.000
17  2022  262805.000


In [150]:
##Train with MLPRegressor

import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import joblib

#SET USING DATA
rice_data=aggregated_rice_data
climate_data=aggregated_climate_data

# 將 '年度' 欄位轉換為整數型態
rice_data['年度'] = rice_data['年度'].astype(int)
climate_data['Year'] = climate_data['Year'].astype(int)

# 合併資料（使用 '年度' 進行合併）
merged_data = pd.merge(rice_data, climate_data, left_on='年度', right_on='Year')

# 檢查合併後的資料
print(merged_data)

# 將合併後的資料存為 CSV 檔案
merged_data.to_csv('merged_data.csv', index=False)



# 選擇特徵
features = ['降水量(mm)']

# 處理缺失值
merged_data = merged_data.dropna(subset=features + ['產量'])

# 分割特徵和目標變數
X = merged_data[features]
y = merged_data['產量']

# 資料標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 分割訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 建立 MLPRegressor 模型
model = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=100, random_state=42)

# 訓練模型
model.fit(X_train, y_train)

# 預測
y_pred = model.predict(X_test)

# 評估模型
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# 儲存模型
joblib.dump(model, 'rice_production_by_MLPRegressor.pkl')


      年度          產量  Year    測站氣壓(hPa)   海平面氣壓(hPa)  測站最高氣壓(hPa)  \
0   2005  208289.704  2005  1008.333333  1017.473611       1027.4   
1   2006  225004.805  2006  1007.975000  1017.473611       1023.9   
2   2007  184585.881  2007  1007.883333  1017.473611       1022.2   
3   2008  219487.599  2008  1008.275000  1017.473611       1021.2   
4   2009  241788.425  2009  1007.700000  1017.473611       1024.9   
5   2010  240184.619  2010  1008.633333  1017.473611       1023.5   
6   2011  256466.058  2011  1008.350000  1017.473611       1022.2   
7   2012  268065.774  2012  1007.400000  1017.473611       1020.6   
8   2013  244610.514  2013  1008.416667  1017.473611       1023.0   
9   2014  262391.030  2014  1008.725000  1017.473611       1022.7   
10  2015  256779.924  2015  1008.950000  1017.473611       1022.4   
11  2016  241900.800  2016  1008.708333  1017.473611       1028.9   
12  2017  266549.996  2017  1009.008333  1017.473611       1025.0   
13  2018  208447.762  2018  1007.7

c:\Users\user\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


['rice_production_by_MLPRegressor.pkl']

In [151]:
##Test with MLPRegressor

import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import joblib

# 載入測試資料
test_data = pd.read_csv('test_data.csv')


# # 處理缺失值
# test_data = test_data.dropna(subset=features + ['產量'])

# 分割特徵和目標變數
X_test = test_data[features]
y_test = test_data['產量']

# 資料標準化
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# 載入已經訓練好的模型
model = joblib.load('rice_production_by_MLPRegressor.pkl')

# 預測
y_pred = model.predict(X_test_scaled)

# 評估模型
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on Test Data: {mse}')

print("RESULT:")
print(y_pred)

Mean Squared Error on Test Data: 559579.4682145234
RESULT:
[8.13548974 8.13548974 8.13548974 8.13548974]
